### Importer les bibliothéque

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import sqlite3
import os
import locale
from datetime import datetime

### Afficher le dataset

In [2]:
df_avis = pd.read_csv('D:/Projet_ Data_Engineer/projet_scraping_trustpilot/data/raw/trustpilot_comments.csv')

In [3]:
df_avis.head()

,Auteur,Date,Note,Commentaire
0,Anonyme,14 novembre 2025,5,"Très bon suivi de Sophie, l'esthéticienne"
1,Anonyme,22 novembre 2025,5,J’ai commencé le programme en septembre 2025 e...
2,Anonyme,22 novembre 2025,5,2 mois et demi de programme cheef entamé et un...
3,Anonyme,21 novembre 2025,5,Arrivée à 108 kg à cause du stress de ne plus ...
4,Anonyme,21 novembre 2025,5,Expérience Cheef très satisfaisante ! Avec Mar...


In [4]:
print("ligne_colonne :" , df_avis.shape , "\n" , "taille_df :" , df_avis.size , "\n" , "colonnes :" , df_avis.columns)

ligne_colonne : (21560, 4) 
 taille_df : 86240 
 colonnes : Index(['Auteur', 'Date', 'Note', 'Commentaire'], dtype='object')


In [5]:
df_avis.dtypes

Auteur         object
Date           object
Note            int64
Commentaire    object
dtype: object

In [6]:
## convertir la colonne Date en format datetime

def convert_french_dates_method1(df):
    """Convertir dates françaises avec locale"""
    
    # Configurer le locale en français
    try:
        locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')  # Linux/Mac
    except:
        try:
            locale.setlocale(locale.LC_TIME, 'fra_fra')  # Windows
        except:
            locale.setlocale(locale.LC_TIME, 'French_France.1252')  # Windows alternatif
    
    # Convertir
    df["Date"] = pd.to_datetime(df["Date"], format="%d %B %Y")
    df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")
    
    return df


In [7]:
df_avis.isnull().sum()

Auteur         0
Date           0
Note           0
Commentaire    0
dtype: int64

### Créer la base SQLite + une table

In [8]:
# Connexion à la base de données dans le dossier existant
conn = sqlite3.connect("../data/database_sqlite/trustpilot_reviews.db")
cur = conn.cursor()
print(" Connexion réussie !")

 Connexion réussie !


In [9]:
# Créer la table
cur.execute('''
    CREATE TABLE IF NOT EXISTS avis (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        auteur TEXT NOT NULL,
        date TEXT NOT NULL,
        note INTEGER NOT NULL,
        commentaire TEXT,
        date_insertion TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
''')

conn.commit()
print(" Table 'avis' créée !")

 Table 'avis' créée !


In [ ]:
# Vérifier que la table existe
cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cur.fetchall()
print(f" Tables dans la base : {tables}")

📋 Tables dans la base : [('avis',), ('sqlite_sequence',)]


In [11]:
#Afficher la structure de la table
cur.execute("PRAGMA table_info(avis)")
colonnes = cur.fetchall()
print("\n Structure de la table 'avis' :")
for col in colonnes:
    print(f"  - {col[1]} ({col[2]})")


 Structure de la table 'avis' :
  - id (INTEGER)
  - auteur (TEXT)
  - date (TEXT)
  - note (INTEGER)
  - commentaire (TEXT)
  - date_insertion (TIMESTAMP)


### Insérer les données depuis votre DataFrame

In [12]:
# Charger votre DataFrame (ajustez le chemin si nécessaire)
df_avis = pd.read_csv("../data/raw/trustpilot_comments.csv", encoding='utf-8-sig')
print(f"\n {len(df_avis)} avis à insérer")

# Insérer les données
for index, row in df_avis.iterrows():
    cur.execute('''
        INSERT INTO avis (auteur, date, note, commentaire)
        VALUES (?, ?, ?, ?)
    ''', (row['Auteur'], row['Date'], row['Note'], row['Commentaire']))

conn.commit()
print(f" {len(df_avis)} avis insérés dans la table !")


 21560 avis à insérer
 21560 avis insérés dans la table !


### VÉRIFICATIONS

In [13]:
# Compter les lignes
cur.execute("SELECT COUNT(*) FROM avis")
total = cur.fetchone()[0]
print(f"\n Nombre d'avis dans la table : {total}")

# Afficher les 5 premiers avis
cur.execute("SELECT * FROM avis LIMIT 5")
rows = cur.fetchall()
print("\n Les 5 premiers avis :")
for row in rows:
    print(f"ID: {row[0]} | {row[1]} | {row[2]} | Note: {row[3]}")
    print(f"   - {row[4][:80]}...")  # Afficher les 80 premiers caractères
    print()

# Distribution des notes
cur.execute('''
    SELECT note, COUNT(*) as nombre
    FROM avis
    GROUP BY note
    ORDER BY note DESC
''')
notes_stats = cur.fetchall()
print(" Distribution des notes :")
for note, nombre in notes_stats:
    print(f"  {note} étoiles : {nombre} avis")

# Fermer la connexion
conn.close()
print("\n Opérations terminées, connexion fermée")


 Nombre d'avis dans la table : 21560

 Les 5 premiers avis :
ID: 1 | Anonyme | 14 novembre 2025 | Note: 5
   - Très bon suivi de Sophie, l'esthéticienne...

ID: 2 | Anonyme | 22 novembre 2025 | Note: 5
   - J’ai commencé le programme en septembre 2025 et moins 10kg, je suis plus que sat...

ID: 3 | Anonyme | 22 novembre 2025 | Note: 5
   - 2 mois et demi de programme cheef entamé et une perte de poids de 12kg. Merci à ...

ID: 4 | Anonyme | 21 novembre 2025 | Note: 5
   - Arrivée à 108 kg à cause du stress de ne plus me sentir bien dans ma vie pro j’a...

ID: 5 | Anonyme | 21 novembre 2025 | Note: 5
   - Expérience Cheef très satisfaisante ! Avec Marine comme diététicienne et accompa...

 Distribution des notes :
  5 étoiles : 19412 avis
  4 étoiles : 2146 avis
  1 étoiles : 2 avis

 Opérations terminées, connexion fermée
